In [1]:
HF_Token = 'YOUR HUGGINGFACE TOKEN'
from huggingface_hub import login
login(token=HF_Token)

Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [2]:
from google.colab import drive
drive.mount('/content/drive')
!mkdir content/drive/MyDrive/StanceEval

Mounted at /content/drive
mkdir: cannot create directory ‘content/drive/MyDrive/StanceEval’: No such file or directory


In [3]:
!git clone https://github.com/NoraAlt/Mawqif-Arabic-Stance.git

Cloning into 'Mawqif-Arabic-Stance'...
remote: Enumerating objects: 14, done.
remote: Counting objects: 100% (11/11), done.
remote: Compressing objects: 100% (11/11), done.
remote: Total 14 (delta 3), reused 2 (delta 0), pack-reused 3
Receiving objects: 100% (14/14), 3.59 MiB | 24.98 MiB/s, done.
Resolving deltas: 100% (3/3), done.


In [4]:
!pip install peft
!pip install trl
!pip install bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.1/199.1 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 19.9 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-many

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 14.3 MB/s eta 0:00:00


In [5]:
import pandas as pd
import numpy as np

In [6]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, TrainingArguments
from peft import prepare_model_for_kbit_training, LoraConfig, get_peft_model
from datasets import load_dataset, Dataset
from trl import SFTTrainer
import torch

In [7]:
train = pd.read_csv('Mawqif-Arabic-Stance/Data/Mawqif_AllTargets_Train.csv')
test = pd.read_csv('Mawqif-Arabic-Stance/Data/Mawqif_AllTargets_Blind Test.csv')

In [8]:
train = train.sort_values(by=['target'])
train = train.reset_index()

In [22]:
mylist = []
inst = ' [INST] <> You are an expert Arabic stance analyzer! <>  \
Consider the subsequent Arabic passage which discusses  '
inst2 =  ' and determine if it is Favor, None, or Against, and return the answer as \
            the corresponding stance label "Favor" or "None" or "Against".\n'
for index, row in train.iterrows():
  string = inst + row['target'] + inst2  + row['text'] + ' [/INST]\n' + str(row['stance']) + ''
  mylist.append(string)

In [25]:
import random
random.shuffle
random.shuffle(mylist)

In [26]:
eval_list = [mylist[i] for i in range(len(mylist)) if i % 5 == 0]
train_list = [mylist[i] for i in range(len(mylist)) if i % 5 > 0]

In [27]:
train_dataset = Dataset.from_dict({"text": train_list})
eval_dataset = Dataset.from_dict({"text": eval_list})
full_dataset = Dataset.from_dict({"text": mylist})

In [16]:
model_name = "meta-llama/Meta-Llama-3-8B-Instruct"
compute_dtype = getattr(torch, "float16")
bnb_config = BitsAndBytesConfig(load_in_4bit=True,
            bnb_4bit_quant_type="nf4",
            bnb_4bit_compute_dtype=compute_dtype,
            bnb_4bit_use_double_quant=True,
)
model = AutoModelForCausalLM.from_pretrained(model_name, device_map={"": 0}, quantization_config=bnb_config)
model = prepare_model_for_kbit_training(model)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  # Means the user did not define a `HF_TOKEN` secret => warn


config.json:   0%|          | 0.00/654 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/187 [00:00<?, ?B/s]

In [17]:
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True, add_eos_token=True,device='cuda')
tokenizer.pad_token = tokenizer.unk_token
tokenizer.padding_side = "left"

tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [18]:
peft_config = LoraConfig(
            lora_alpha=16,
            lora_dropout=0.05,
            r=16,
            bias="none",
            task_type="SEQ_CLS",
            target_modules= ["down_proj","up_proj","gate_proj"]
)

In [19]:
peft_config = LoraConfig(
            lora_alpha=16,
            lora_dropout=0.05,
            r=64,
            bias="none",
            task_type="SEQ_CLS",
            target_modules= ["q_proj","up_proj","o_proj","k_proj","down_proj","gate_proj","v_proj"]
)

In [20]:
training_arguments = TrainingArguments(
        output_dir="./results/",
        evaluation_strategy="steps",
        optim="paged_adamw_8bit",
        save_steps=50,
        log_level="debug",
        logging_steps=50,
        learning_rate=1e-4,
        eval_steps=50,
        fp16=True,
        do_eval=True,
        per_device_train_batch_size=1,
        per_device_eval_batch_size=100,
        #gradient_accumulation_steps=1,
        warmup_steps=10,
        max_steps=5600,
        lr_scheduler_type="linear"
)

In [28]:
tokenizer.pad_token = tokenizer.eos_token
trainer = SFTTrainer(
        model=model,
        train_dataset=train_dataset,
        eval_dataset=eval_dataset,
        peft_config=peft_config,
        dataset_text_field="text",
        max_seq_length=120,
        tokenizer=tokenizer,
        args=training_arguments
)

trainer.train()

PyTorch: setting up devices


Map:   0%|          | 0/2801 [00:00<?, ? examples/s]

Map:   0%|          | 0/701 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:318: UserWarning: You passed a tokenizer with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precision. You might consider adding `tokenizer.padding_side = 'right'` to your code.
  warnings.warn(
max_steps is given, it will override any value given in num_train_epochs
Using auto half precision backend
Currently training with a batch size of: 1
***** Running training *****
  Num examples = 2,801
  Num Epochs = 2
  Instantaneous batch size per device = 1
  Total train batch size (w. parallel, distributed & accumulation) = 1
  Gradient Accumulation steps = 1
  Total optimization steps = 5,600
  Number of trainable parameters = 167,772,160
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The de

Step,Training Loss,Validation Loss
50,2.395200,1.641574
100,1.442800,1.595816
150,1.491500,1.560811
200,1.508400,1.544421
250,1.475800,1.547366
300,1.420500,1.528510
350,1.506200,1.518409
400,1.393100,1.526596
450,1.433200,1.508729
500,1.466200,1.508986


***** Running Evaluation *****
  Num examples = 701
  Batch size = 100
Saving model checkpoint to ./results/checkpoint-50
loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--meta-llama--Meta-Llama-3-8B-Instruct/snapshots/c4a54320a52ed5f88b7a2f84496903ea4ff07b45/config.json
Model config LlamaConfig {
  "architectures": [
    "LlamaForCausalLM"
  ],
  "attention_bias": false,
  "attention_dropout": 0.0,
  "bos_token_id": 128000,
  "eos_token_id": 128009,
  "hidden_act": "silu",
  "hidden_size": 4096,
  "initializer_range": 0.02,
  "intermediate_size": 14336,
  "max_position_embeddings": 8192,
  "model_type": "llama",
  "num_attention_heads": 32,
  "num_hidden_layers": 32,
  "num_key_value_heads": 8,
  "pretraining_tp": 1,
  "rms_norm_eps": 1e-05,
  "rope_scaling": null,
  "rope_theta": 500000.0,
  "tie_word_embeddings": false,
  "torch_dtype": "bfloat16",
  "transformers_version": "4.40.2",
  "use_cache": true,
  "vocab_size": 128256
}

tokenizer con

TrainOutput(global_step=5600, training_loss=1.2347327498027256, metrics={'train_runtime': 5395.792, 'train_samples_per_second': 1.038, 'train_steps_per_second': 1.038, 'total_flos': 2.878577946776371e+16, 'train_loss': 1.2347327498027256, 'epoch': 1.9992859692966798})

In [29]:
test['stance'] = ''

In [ ]:
for index, row in test.iterrows():
  #if len(row['target_msa']) > 1:
  #  continue
  print(index)
  prompt = inst + row['target'] + inst2  + row['text'] + ' [/INST]\n'
  tokenized_input = tokenizer(prompt, return_tensors="pt")
  input_ids = tokenized_input["input_ids"].cuda()

  generation_output = model.generate(
        input_ids=input_ids,
        num_beams=6,
        return_dict_in_generate=True,
        output_scores=True,
        max_new_tokens=30
  )
  ans = tokenizer.decode(generation_output.sequences[0], skip_special_tokens=True).split('[/INST]')[1]
  ans = ans.split()[0].replace('.','')
  test.at[index, 'stance'] = ans
  print(ans,row['text'])
  if index % 10 == 0:
    test.to_csv('drive/MyDrive/StanceEval/test.csv',index=False)
    test['stance'].to_csv('drive/MyDrive/StanceEval/predictions.csv',index=False)
    #break
test.to_csv('drive/MyDrive/StanceEval/test.csv',index=False)
test['stance'].to_csv('drive/MyDrive/StanceEval/predictions.csv',index=False)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


0
Against الوضع صار بالسعوديه يفشل لا عاد فيه شركات صاحيه ولا مشاهير صاحين.  الوضع الحالي اقبح صور تسليع المرأه تحت مسمى تمكين المرأه!  #مقاطعة_شاهي_خدير


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


1


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


...

Against ريم النجم:  تطعيم كورونا راح يحولكم زومبي ! 
616


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Favor البيانات هي العنصر الرئيسي و المهم جداً في رحلة التحول لإقتصاد رقمي  ربما حظي ببعض الاهتمام الا أنه يحتاج للمزيد من الاهتمام الذي يرتقي لطموحاتنا لتحقيق اقتصاد رقمي حقيقي    #كورونا #جمعية_انتاج #التحول_الرقمي #COVID19 #DigitalTransformation #STS  URL
617


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Favor  تطعيم انفلونزا مناعة مؤقته تطعيم كورونا مناعة مؤقته  تفيد مرضى القلب الربو كبار السن تفيد للجميع  ان شاء الله
618


In [ ]:
test

In [ ]:
from google.colab import runtime
runtime.unassign()